In [ ]:
# XML虽然比JSON复杂，在Web中应用也不如以前多了，不过仍有很多地方在用，所以，有必要了解如何操作XML。
# DOM vs SAX
# 操作XML有两种方法：DOM和SAX。DOM会把整个XML读入内存，解析为树，因此占用内存大，
# 解析慢，优点是可以任意遍历树的节点。SAX是流模式，边读边解析，占用内存小，解析快，缺点是我们需要自己处理事件。
# 正常情况下，优先考虑SAX，因为DOM实在太占内存。
# 在Python中使用SAX解析XML非常简洁，通常我们关心的事件是start_element，end_element和char_data，
# 准备好这3个函数，然后就可以解析xml了。
# 举个例子，当SAX解析器读到一个节点时：

# <a href="/">python</a>

# 会产生3个事件：

# start_element事件，在读取<a href="/">时；
# char_data事件，在读取python时；
# end_element事件，在读取</a>时。

In [1]:
# 用代码实验一下：

from xml.parsers.expat import ParserCreate

class DefaultSaxHandler(object):
    def start_element(self, name, attrs):
        print('sax:start_element: %s, attrs: %s' % (name, str(attrs)))

    def end_element(self, name):
        print('sax:end_element: %s' % name)

    def char_data(self, text):
        print('sax:char_data: %s' % text)

xml = r'''<?xml version="1.0"?>
<ol>
    <li><a href="/python">Python</a></li>
    <li><a href="/ruby">Ruby</a></li>
</ol>
'''

handler = DefaultSaxHandler()
parser = ParserCreate()
parser.StartElementHandler = handler.start_element
parser.EndElementHandler = handler.end_element
parser.CharacterDataHandler = handler.char_data
parser.Parse(xml)


sax:start_element: ol, attrs: {}
sax:char_data: 

sax:char_data:     
sax:start_element: li, attrs: {}
sax:start_element: a, attrs: {'href': '/python'}
sax:char_data: Python
sax:end_element: a
sax:end_element: li
sax:char_data: 

sax:char_data:     
sax:start_element: li, attrs: {}
sax:start_element: a, attrs: {'href': '/ruby'}
sax:char_data: Ruby
sax:end_element: a
sax:end_element: li
sax:char_data: 

sax:end_element: ol


1

In [ ]:
L = []
L.append(r'<?xml version="1.0"?>')
L.append(r'<root>')
L.append(encode('some & data'))
L.append(r'</root>')
return ''.join(L)
# 如果要生成复杂的XML呢？建议你不要用XML，改成JSON。

NameError: name 'encode' is not defined

In [5]:
# 解析XML时，注意找出自己感兴趣的节点，响应事件时，把节点数据保存起来。解析完毕后，就可以处理数据。

# 请利用SAX编写程序解析WeatherAPI的XML格式的天气预报，获取天气预报：
from xml.parsers.expat import ParserCreate
from urllib import request

class WeatherSaxHandler:
    def __init__(self):
        self.weather_data = {
            'city': '',
            'weather': {
                'condition': '',
                'temperature': '',
                'wind': ''
            }
        }
        self.current_element = ''
        self.current_data = ''
    
    def start_element(self, name, attrs):
        self.current_element = name
    
    def end_element(self, name):
        if name == 'name':
            self.weather_data['city'] = self.current_data.strip()
        elif name == 'text':
            self.weather_data['weather']['condition'] = self.current_data.strip()
        elif name == 'temp_c':
            self.weather_data['weather']['temperature'] = self.current_data.strip()
        elif name == 'wind_kph':
            self.weather_data['weather']['wind'] = self.current_data.strip()
        
        self.current_data = ''
    
    def char_data(self, data):
        self.current_data += data

def parseXml(xml_str):
    handler = WeatherSaxHandler()
    parser = ParserCreate()
    parser.StartElementHandler = handler.start_element
    parser.EndElementHandler = handler.end_element
    parser.CharacterDataHandler = handler.char_data
    
    parser.Parse(xml_str)
    
    return handler.weather_data

# 测试:
URL = 'https://api.weatherapi.com/v1/current.xml?key=b4e8f86b44654e6b86885330242207&q=Beijing&aqi=no'

with request.urlopen(URL, timeout=4) as f:
    data = f.read()

result = parseXml(data.decode('utf-8'))
print(f"City: {result['city']}")
print(f"Weather Condition: {result['weather']['condition']}")
print(f"Temperature: {result['weather']['temperature']}°C")
print(f"Wind Speed: {result['weather']['wind']} kph")

assert result['city'] == 'Beijing'
print("测试通过!")

City: Beijing
Weather Condition: Overcast
Temperature: 12.3°C
Wind Speed: 7.2 kph
测试通过!
